In [1]:
import pandas as pd
import gzip
import xmltodict

In [2]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict


In [6]:
chemin_612245 = '../../fichiers20/todo_xml_20/612450.xml.gz'

In [7]:
parse1 = parse_fichier(chemin_612245)

In [8]:
parse1

{'DocumentBudgetaire': {'@xmlns': 'http://www.minefi.gouv.fr/cp/demat/docbudgetaire',
  'VersionSchema': {'@V': '86'},
  'BlocEditeur': {'@xmlns:bl': 'http://www.berger-levrault.fr/demat/outils',
   'CodeEditeur': {'@V': 'Berger-Levrault'},
   'bl:ProduitGF': {'@V': 'e.gf évolution - 15.10.00.01'},
   'bl:EchangeXML': {'@V': '2020.07'}},
  'VersionOutil': [{'@outil': 'DSC_COMMON', '@version': '27'},
   {'@outil': 'REMAT', '@version': '3'}],
  'Scellement': {'@md5': '03C1A89780727DE3D4AA129CE4D2BC9E',
   '@sha1': '5CF837754DD96DB2F18186742AE1E32ABB71EE1B',
   '@date': '2020-08-03T12:35:01.582+02:00'},
  'EnTeteDocBudgetaire': {'DteStr': {'@V': '2020-08-03'},
   'LibellePoste': {'@V': 'TRESORERIE PRINCIPALE'},
   'IdPost': {'@V': '009016'},
   'LibelleColl': {'@V': 'COMMUNAUTE DE COMMUNES COUSERANS PYRENEES'},
   'IdColl': {'@V': '20006794000185'},
   'NatCEPL': {'@V': 'Communauté de commune à fiscalité additionnelle'},
   'Departement': {'@V': '09'}},
  'Budget': {'EnTeteBudget': {'Libe

In [9]:
dico1 = parse1['DocumentBudgetaire']['Budget']['LigneBudget']

In [31]:
mtdict = 0
mtlist = 0
mtnan = 0
ldict = []
llist = []
for i in dico1 : 
    type_m = i.get('MtSup')  # Permet de connaitre le type de MtSup
    if isinstance(type_m, dict) : 
        mtdict += 1 
        ldict.append(i)
    elif isinstance(type_m, list) : 
        mtlist +=1 
        dico12.append(i)
    else : 
        mtnan +=1

print('dict : ', mtdict)
print('list : ', mtlist)
print('nan : ', mtnan)

dict :  24
list :  6
nan :  75


In [14]:
for i in ldict : 
 type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
 dict_mtsup = type_mtsup
 i[dict_mtsup.get('@Code', '')] = {'@V' : dict_mtsup.get('@V', '')}

df1_dict = pd.DataFrame(ldict)
df1_dict.columns

Index(['Nature', 'ContNat', 'ArtSpe', 'CodRD', 'MtPrev', 'MtReal', 'OpBudg',
       'MtSup', 'CaracSup', 'ProdChaRat', 'LibCpte', 'Operation', 'ContOp',
       'MtPropNouv', 'CredOuv', 'MtRAR3112', 'MtOpeInfo', 'MtOpeCumul'],
      dtype='object')

In [ ]:
df1_dict[['MtOpeInfo', 'MtOpeCumul']]

In [ ]:
dico11 = dico1.copy()
for i in dico11 : 
 type_m = i.get('MtSup')
 if type_m is None : 
  pass 
 else : 
  if isinstance(type_m, dict) : 
   i[type_m.get('@Code', '')] = {'@V' : type_m.get('@V', '')}

df1_full_dict = pd.DataFrame(dico11)
df1_full_dict_mt = df1_full_dict[['MtSup', 'MtOpeInfo', 'MtOpeCumul', 'ProdChaRat']]
#pd.set_option('display.max_rows', 500)
#pd.reset_option('all')
df1_full_dict_mt

Actuellement, on a un df1 qui a pu travailler tout les mtsup dictionnaires. 

In [35]:
dico12 = dico1.copy()

In [36]:
type(dico12)    #list
type(dico12[1]) #dict

dict

In [52]:
for i in dico12 : 
 type_m = i.get('MtSup')
 if type_m is None : 
  pass 
 else : 
  #if isinstance(type_m, dict) : 
   #i[type_m.get('@Code', '')] = {'@V' : type_m.get('@V', '')}
  if isinstance(type_m, list) :
    #print(len(type_m)) On a bien les 6 
    for z in type_m : #z c'est un dict contenant un V et un Code 
      #type_m[z.get('@Code', '')] = {'@V' : z.get('@V', '')}
      i[z.get('@Code', '')] = {'@V' : z.get('@V', '')}

In [60]:
df12 = pd.DataFrame(dico12)
pd.set_option('display.max_rows', 500)
df121 = df12[['MtSup', 'MtOpeInfo', 'MtOpeCumul', 'ProdChaRat']]
df121[~df121['MtSup'].isna()]

,MtSup,MtOpeInfo,MtOpeCumul,ProdChaRat
2,"{'@V': '8332.84', '@Code': 'ProdChaRat'}",NaN,NaN,{'@V': '8332.84'}
24,"{'@V': '1901.52', '@Code': 'ProdChaRat'}",NaN,NaN,{'@V': '1901.52'}
35,"{'@V': '13624.58', '@Code': 'ProdChaRat'}",NaN,NaN,{'@V': '13624.58'}
61,"{'@V': '219903.87', '@Code': 'ProdChaRat'}",NaN,NaN,{'@V': '219903.87'}
76,"{'@V': '20000.00', '@Code': 'MtOpeInfo'}",{'@V': '20000.00'},NaN,NaN
77,"[{'@V': '720.00', '@Code': 'MtOpeCumul'}, {'@V...",{'@V': '180000.00'},{'@V': '720.00'},NaN
78,"{'@V': '150000.00', '@Code': 'MtOpeInfo'}",{'@V': '150000.00'},NaN,NaN
79,"[{'@V': '55674.41', '@Code': 'MtOpeCumul'}, {'...",{'@V': '150000.00'},{'@V': '55674.41'},NaN
80,"{'@V': '720.00', '@Code': 'MtOpeCumul'}",NaN,{'@V': '720.00'},NaN
81,"[{'@V': '19213.00', '@Code': 'MtOpeCumul'}, {'...",{'@V': '50000.00'},{'@V': '19213.00'},NaN
